#### Author: Thejasvi Beleyur
#### Last updated: 2020-03-17

This notebook will detail all the steps required to get matching audio snippets for the annotations of bat flights made for 2018-08-19 between 02:00-03:00 hours. 
This notebook is a copy of 'Matching annotations to audio 2018-08-20 0111-0200.ipynb'. 



### Steps carried out before hand:
1. Generating the video sync for the video file *OrlovaChukaDome_02_20180820_01.11.28-02.00.00[R][@42dd][1]....avi* with frame-wise light intensities and time stamps

2. Video annotations of the horseshoebats flying done by  Aditya Kanjoor. 


In [1]:
import matplotlib.pyplot as plt
import natsort
import numpy as np
import pandas as pd 
from tqdm import tqdm

package_folder = '../../../bin/'
import os
import sys 
sys.path.append(package_folder) # include the modules in the outer folder in the search path
from process_video_annotations import video_sync_over_annotation_block



In [2]:
%matplotlib notebook

In [3]:
# loading the annotations
annotations = pd.read_csv('annotations/corrected_HBC_video_annotations_Aditya/2018-08-20_0200-0300_Aditya_corrected.csv')

In [4]:
annotations.head()

,video_path,annotation_id,Sampling minute,timestamp from camera,start_timestamp,start_framenumber,end_timestamp,end_framenumber,no. of flying bats,bat behaviour,"strength (1 - strong, 0-weak)","weak in (Behaviour Be, Number Bn)",Comments,Details on the comment,annotator
0,OrlovaChukaDome_02_20180820_02.00.00-03.00.00[...,Aditya_2018-08-20_0200-0300_1,1,2,2018-08-20 02:01:00,24,2018-08-20 02:01:01,23,1,NaN,1,NaN,NaN,NaN,Aditya
1,OrlovaChukaDome_02_20180820_02.00.00-03.00.00[...,Aditya_2018-08-20_0200-0300_2,5,2,2018-08-20 02:05:03,11,2018-08-20 02:05:04,10,1,NaN,1,NaN,NaN,NaN,Aditya
2,OrlovaChukaDome_02_20180820_02.00.00-03.00.00[...,Aditya_2018-08-20_0200-0300_3,5,2,2018-08-20 02:05:10,8,2018-08-20 02:05:11,3,1,NaN,1,NaN,NaN,NaN,Aditya
3,OrlovaChukaDome_02_20180820_02.00.00-03.00.00[...,Aditya_2018-08-20_0200-0300_4,5,2,2018-08-20 02:05:52,6,2018-08-20 02:05:52,21,1,NaN,1,NaN,NaN,NaN,Aditya
4,OrlovaChukaDome_02_20180820_02.00.00-03.00.00[...,Aditya_2018-08-20_0200-0300_5,6,2,2018-08-20 02:06:48,10,2018-08-20 02:06:49,12,1,NaN,1,NaN,NaN,NaN,Aditya


In [5]:
annotations.shape

(97, 15)

In [6]:
annots_w_bats = annotations[annotations['no. of flying bats']>0].reset_index(drop=True)

In [7]:
annotations.shape

(97, 15)

In [8]:
annots_w_bats.shape

(97, 15)

In [9]:
annots_w_bats.head()

,video_path,annotation_id,Sampling minute,timestamp from camera,start_timestamp,start_framenumber,end_timestamp,end_framenumber,no. of flying bats,bat behaviour,"strength (1 - strong, 0-weak)","weak in (Behaviour Be, Number Bn)",Comments,Details on the comment,annotator
0,OrlovaChukaDome_02_20180820_02.00.00-03.00.00[...,Aditya_2018-08-20_0200-0300_1,1,2,2018-08-20 02:01:00,24,2018-08-20 02:01:01,23,1,NaN,1,NaN,NaN,NaN,Aditya
1,OrlovaChukaDome_02_20180820_02.00.00-03.00.00[...,Aditya_2018-08-20_0200-0300_2,5,2,2018-08-20 02:05:03,11,2018-08-20 02:05:04,10,1,NaN,1,NaN,NaN,NaN,Aditya
2,OrlovaChukaDome_02_20180820_02.00.00-03.00.00[...,Aditya_2018-08-20_0200-0300_3,5,2,2018-08-20 02:05:10,8,2018-08-20 02:05:11,3,1,NaN,1,NaN,NaN,NaN,Aditya
3,OrlovaChukaDome_02_20180820_02.00.00-03.00.00[...,Aditya_2018-08-20_0200-0300_4,5,2,2018-08-20 02:05:52,6,2018-08-20 02:05:52,21,1,NaN,1,NaN,NaN,NaN,Aditya
4,OrlovaChukaDome_02_20180820_02.00.00-03.00.00[...,Aditya_2018-08-20_0200-0300_5,6,2,2018-08-20 02:06:48,10,2018-08-20 02:06:49,12,1,NaN,1,NaN,NaN,NaN,Aditya


### Carrying out the common-fps conversion:


In [10]:
# load the video sync file with corrected timestamps
video_sync_file = 'videosync_OrlovaChukaDome_01_20180820_02.00.00-03.00.00[R][@45f1][2].avi_[timestamp_corrected]_[signal_checked].csv'
path_to_videosync = os.path.join('raw_sync','2018-08-19',
                                 video_sync_file)
video_sync = pd.read_csv(path_to_videosync)

In [11]:
video_sync.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,frame_number,led_intensity,timestamp,timestamp_verified,user_suggestion
0,0,0,0,1,1,2.034884,2018-08-20 02:00:00,2018-08-20 02:00:00,maybeokay
1,1,1,1,2,2,2.174419,2018-08-20 02:00:00,2018-08-20 02:00:00,maybeokay
2,2,2,2,3,3,2.174419,2018-08-20 02:00:00,2018-08-20 02:00:00,maybeokay
3,3,3,3,4,4,2.197674,2018-08-20 02:00:00,2018-08-20 02:00:00,maybeokay
4,4,4,4,5,5,2.197674,2018-08-20 02:00:00,2018-08-20 02:00:00,maybeokay


In [12]:
kwargs = {'timestamp_pattern': '%Y-%m-%d %H:%M:%S'}
kwargs['min_fps']= 21 # Hz
kwargs['min_durn'] = 45.0 # seconds 
kwargs['common_fps'] = 25 # Hz
kwargs['output_folder'] = './common_fps/Aditya_annotations/2018-08-19/'

unprocessed_annotations = []

for i, each_annotation in tqdm(annots_w_bats.iterrows()):
    try:
        success = video_sync_over_annotation_block(each_annotation,video_sync, 
                                                     **kwargs)
    except :
        unprocessed_annotations.append(i)
        print('Unable to process annotation: %s'%annots_w_bats['annotation_id'][i])


0it [00:00, ?it/s]../../../bin/process_video_annotations.py:389: UserWarning: Some timestamps below min fps!
  warnings.warn('Some timestamps below min fps!')
../../../bin/process_video_annotations.py:310: UserWarning: FPS fell below the minimum required - ignoring this annotation
  warnings.warn('FPS fell below the minimum required - ignoring this annotation')
../../../bin/process_video_annotations.py:115: UserWarning: could not bring above annotation to common fps,                      please check a) if the corresponding audio file is present                        b) if the ON/OFF of the light signal is clean                           
  ')
1it [00:07,  7.01s/it]

('timestamps below min fps: ', array(['2018-08-20 02:01:31'], dtype=object))
video_path                           OrlovaChukaDome_02_20180820_02.00.00-03.00.00[...
annotation_id                                            Aditya_2018-08-20_0200-0300_1
Sampling minute                                                                      1
timestamp from camera                                                                2
start_timestamp                                                    2018-08-20 02:01:00
start_framenumber                                                                   24
end_timestamp                                                      2018-08-20 02:01:01
end_framenumber                                                                     23
no. of flying bats                                                                   1
bat behaviour                                                                      NaN
strength (1 - strong, 0-weak)                        

/home/tbeleyur/anaconda2/envs/horseshoebat/lib/python2.7/site-packages/scipy/signal/signaltools.py:2223: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/home/tbeleyur/anaconda2/envs/horseshoebat/lib/python2.7/site-packages/scipy/signal/signaltools.py:2225: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/home/tbeleyur/anaconda2/envs/horseshoebat/lib/python2.7/site-packages/scipy/signal/signaltools.py:2233: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` 

Saved annotation :Aditya_2018-08-20_0200-0300_2


3it [00:23,  7.48s/it]

...

Saved annotation :Aditya_2018-08-20_0200-0300_95


../../../bin/process_video_annotations.py:289: UserWarning: The following timestamps are missing in the annotation
  stacklevel=1)
96it [15:03,  7.91s/it]

('2018-08-20 03:00:00', '...to...', '2018-08-20 03:00:05')
video_path                           OrlovaChukaDome_02_20180820_02.00.00-03.00.00[...
annotation_id                                           Aditya_2018-08-20_0200-0300_96
Sampling minute                                                                     59
timestamp from camera                                                                2
start_timestamp                                                    2018-08-20 02:59:20
start_framenumber                                                                    5
end_timestamp                                                      2018-08-20 02:59:20
end_framenumber                                                                     14
no. of flying bats                                                                   1
bat behaviour                                                                      NaN
strength (1 - strong, 0-weak)                                          

97it [15:07,  6.52s/it]

('2018-08-20 03:00:00', '...to...', '2018-08-20 03:00:05')
video_path                           OrlovaChukaDome_02_20180820_02.00.00-03.00.00[...
annotation_id                                           Aditya_2018-08-20_0200-0300_97
Sampling minute                                                                     59
timestamp from camera                                                                2
start_timestamp                                                    2018-08-20 02:59:20
start_framenumber                                                                   21
end_timestamp                                                      2018-08-20 02:59:23
end_framenumber                                                                      2
no. of flying bats                                                                   2
bat behaviour                                                                        T
strength (1 - strong, 0-weak)                                          

### Matching Audio from the common_fps files

In [13]:
### Finding the matching audio snippet !! 
import glob 
import soundfile as sf
from audio_for_videoannotation import match_video_sync_to_audio, search_for_best_fit

#all_commonfps = glob.glob('common_fps_video_sync*') # get all the relevant common_fps_sync files
all_commonfps = glob.glob('common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_020*')
audio_folder = '/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/audio_non_spikey/2018-08-19/ch1/' # the current folder

# the output folders
audiosync_folder = './sync_audio/'
audioannotation_folder = './annotation_audio/'
fs = 250000 # change according to the recording sampling rate in Hz!! 


In [14]:
all_ccs = []
files_to_run = sorted(all_commonfps)

def generate_matching_audio(each_commonfps, audio_folder, 
                           **kwargs):
    '''
    '''
    print(each_commonfps)
    try:
        video_sync = pd.read_csv(each_commonfps)
        best_audio, syncblock_audio, crosscoef = match_video_sync_to_audio(video_sync, audio_folder, 
                                                                           **kwargs)
        all_ccs.append(crosscoef)
        fname  = os.path.split(each_commonfps)[-1]
        annotation_id = '-'.join(os.path.split(fname)[-1].split('common_fps_video_sync')[-1:])[:-4]

        audiosync_folder = kwargs.get('audiosync_folder', 'sync_audio')
        audioannotation_folder = kwargs.get('audioannotation_folder', 'annotation_audio')

    
        audio_file_base_name = annotation_id+'.WAV'
        sf.write(audiosync_folder+'matching_sync_'+audio_file_base_name, syncblock_audio,fs)
        sf.write(audioannotation_folder+'matching_annotaudio_'+audio_file_base_name, best_audio,fs)
        return(True)
    except:
        print('Could not save ', each_commonfps)
        return(False)



In [15]:
glob.glob(audio_folder+'*.WAV')

['/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/audio_non_spikey/2018-08-19/ch1/non_spikey_T0000007.WAV',
 '/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/audio_non_spikey/2018-08-19/ch1/non_spikey_T0000008.WAV',
 '/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/audio_non_spikey/2018-08-19/ch1/non_spikey_T0000009.WAV',
 '/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/audio_non_spikey/2018-08-19/ch1/non_spikey_T0000010.WAV',
 '/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/audio_non_spikey/2018-08-19/ch1/non_spikey_T0000011.WAV',
 '/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/audio_non_spikey/2018-08-19/ch1/non_spikey_T0000012.WAV',
 '/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/audio_non_spikey/2018-08-19/ch1/non_spikey_T0000013.WAV',
 '/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/au

In [16]:
natsort.natsorted(os.listdir(audio_folder))

['make_non_spikey_2018-08-20_audio_conversion.py',
 'non_spikey_T0000001.WAV',
 'non_spikey_T0000002.WAV',
 'non_spikey_T0000003.WAV',
 'non_spikey_T0000004.WAV',
 'non_spikey_T0000005.WAV',
 'non_spikey_T0000006.WAV',
 'non_spikey_T0000007.WAV',
 'non_spikey_T0000008.WAV',
 'non_spikey_T0000009.WAV',
 'non_spikey_T0000010.WAV',
 'non_spikey_T0000011.WAV',
 'non_spikey_T0000012.WAV',
 'non_spikey_T0000013.WAV',
 'non_spikey_T0000014.WAV',
 'non_spikey_T0000015.WAV',
 'non_spikey_T0000016.WAV',
 'non_spikey_T0000017.WAV',
 'non_spikey_T0000018.WAV',
 'non_spikey_T0000019.WAV',
 'non_spikey_T0000020.WAV',
 'non_spikey_T0000021.WAV',
 'non_spikey_T0000022.WAV',
 'non_spikey_T0000023.WAV',
 'non_spikey_T0000024.WAV',
 'non_spikey_T0000025.WAV',
 'non_spikey_T0000026.WAV',
 'non_spikey_T0000027.WAV',
 'non_spikey_T0000028.WAV',
 'non_spikey_T0000029.WAV',
 'non_spikey_T0000030.WAV',
 'non_spikey_T0000031.WAV',
 'non_spikey_T0000032.WAV',
 'non_spikey_T0000033.WAV',
 'non_spikey_T0000034.WAV

In [17]:
all_commonfps

['common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_2.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_55.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_68.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_53.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_52.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_3.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_19.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_60.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_6.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_8.

In [18]:
audio_folder

'/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/audio_non_spikey/2018-08-19/ch1/'

In [19]:
len(all_commonfps)

56

In [20]:
natsort.natsorted(all_commonfps)

['common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_2.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_3.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_5.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_6.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_7.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_8.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_11.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_13.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_16.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_17.cs

### A slight change in how annotations are matched to audio. 
4My previous approach to matching annotations to audio was to give *all* audio files in the recording session and let the program search for the best fit. This basically meant that even though the annotation referred only to the first 5 minutes, the *whole* session would be searched. This approach is time consuming. Ever since I tried giving only 10 minutes of audio to be searched for the 02:00-02:10 on 2018-08-19, I realised it takes only about a minute to search through 10 minutes of audio. This means if I break the ~40minutes of audio into four 10minute chunks, then the program will only take one minute per annotation! This should mean, even if there are 120 annotations - it'll take upto ~2hours to complete. I'm wondering why I didn't think of this earlier now....

In [53]:
sorted_commonfps = natsort.natsorted(all_commonfps)

common_fps_chunks = np.array_split(np.array(sorted_commonfps),4) # split the commonfps files into 10 minutes bins


In [54]:
# get annotation number of first and last annoration for each chunks
for each in common_fps_chunks:
    print(each[0] + ',....' + each[-1] +'\n' )

common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_2.csv,....common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_23.csv

common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_24.csv,....common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_48.csv

common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_50.csv,....common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_67.csv

common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_68.csv,....common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_95.csv



In [56]:
file_ranges_for_chunks = [('non_spikey_T0000035','non_spikey_T0000065'),
                          ('non_spikey_T0000055','non_spikey_T0000075'),
                          ('non_spikey_T0000065','non_spikey_T0000085'),
                          ('non_spikey_T0000075','non_spikey_T0000100'),] # the corresponding audio file range for each 10 minute bin


In [57]:
for commonfps_set, file_range in zip(common_fps_chunks, file_ranges_for_chunks):
    for each in commonfps_set:
        generate_matching_audio(each, audio_folder, audio_sync_spikey=False, 
                                file_subset=file_range,
                                audio_fileformat='*.WAV',
                              audiosync_folder='sync_audio/Aditya_annotations/',
                               audioannotation_folder='annotation_audio/Aditya_annotations/')

common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0200-0300_2.csv
('video_fps obtained is :', 25)
Did not find user-provided sample rate - getting it from first file that matches format!
('sampling rate is : ', 250000)
.....finding best audio segment.....


  0%|          | 0/30 [00:00<?, ?it/s]

('max mins for upsampled video', -0.7369167835032189, 1.0)
('max mins', -0.9936673943340251, 1.0)


  3%|▎         | 1/30 [00:05<02:29,  5.15s/it]

('max mins', -0.9884825570880318, 1.0)


...

('max mins', -1.0, 0.986671678438572)


 96%|█████████▌| 24/25 [01:47<00:04,  4.70s/it]

('max mins', -1.0, 0.9799961304452885)


100%|██████████| 25/25 [01:52<00:00,  4.69s/it]


('Best file pairs are: ', 'non_spikey_T0000092.WAV*non_spikey_T0000093.WAV')
Start and end indices around peak:                  start index: 16788669		          end index: 28288669                  total samples:30015488
('AV Sync was above threshold: ', 0.8304967392428789)


In [58]:
import datetime as dt

In [59]:
print('Notebook saved: ', dt.datetime.now())

('Notebook saved: ', datetime.datetime(2020, 3, 18, 9, 58, 43, 813900))
